Going to play around with the lightgb library for training

In [1]:
# /Users/teliov/TUD/Thesis/Medvice/Notebooks/data/04_06_new_data/data/split
# So we can use the *thesislib* package
import sys
import os

module_path = os.path.abspath("..")

if module_path not in sys.path:
    sys.path.append(module_path)

In [23]:
from thesislib.utils.ml import models
from thesislib.utils import pathutils, stringutils

In [5]:
import pandas as pd
import numpy as np
import json

In [24]:
symptom_db_file = pathutils.get_data_file("definitions/symptoms_db.json")
with open(symptom_db_file) as fp:
    symptom_db = json.load(fp)
num_symptoms = len(symptom_db)

symptom_keys = sorted(symptom_db.keys())
symptom_names = [stringutils.slugify(symptom_db[key]) for key in symptom_keys]


In [9]:
data_file = pathutils.get_data_file("06_01_lightgbm/test.csv")
df = pd.read_csv(data_file, index_col="Index")

/Users/teliov/Library/anaconda3/envs/ml/lib/python3.6/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [12]:
label_values = df.LABEL.values

In [13]:
ordered_keys = ['GENDER', 'RACE', 'AGE', 'SYMPTOMS']
df = df[ordered_keys]

In [14]:
sparsifier = models.ThesisSymptomSparseMaker(num_symptoms=num_symptoms)

In [15]:
data_csc = sparsifier.fit_transform(df)

In [16]:
data_csc.shape

(1130087, 379)

In [26]:
feature_names = ['gender', 'race', 'age'] + symptom_names
categorical_features  = ['gender', 'race'] + symptom_names

In [27]:
from sklearn.model_selection import StratifiedShuffleSplit

In [28]:
split_t = StratifiedShuffleSplit(n_splits=1, test_size=0.2)
train_data = None
train_labels = None
test_data = None
test_labels = None
for train_index, test_index in split_t.split(data_csc, label_values):
    train_data = data_csc[train_index]
    train_labels = label_values[train_index]
    test_data = data_csc[test_index]
    test_labels = label_values[test_index]

In [30]:
from lightgbm import LGBMClassifier

In [31]:
clf = LGBMClassifier(n_estimators=10)

In [32]:
clf = clf.fit(train_data, train_labels)

KeyboardInterrupt: 